<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_7(Model_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -U google-generativeai tavily-python pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.0 MB/s eta 0:00:00


In [14]:
import os
import io
import json
import datetime
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
from IPython.display import display, Markdown

# --- API CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY_API_KEY in Colab Secrets.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest')
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("Setup complete.")

Setup complete.


In [44]:
# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:15] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

# --- HELPER: PDF TEXT EXTRACTOR ---
def extract_pdf_text(uploaded_file_bytes):
    try:
        pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file_bytes))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# --- STRICT AGENT STEPS ---

def step_1_generate_questions(input_text, mode):
    """Generates 3 questions based on mode."""
    print(f"\nStep 1: Generating 3 sub-questions (Mode: {mode})...")

    if mode == "topic":
        prompt = f"""
        Task: Break down the topic "{input_text}" into exactly 3 searchable questions.
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        Task: Read the following paper excerpt and generate 3 search questions to VALIDATE the claims or check for newer updates.
        Paper Context: {input_text[:10000]}
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]
    return questions[:3]

def step_2_get_responses(questions):
    """Retrieves short responses (Max 1 result per question)."""
    print(f"Step 2: Retrieving short responses from the web...")
    q_and_a_results = []

    for i, q in enumerate(questions, 1):
        print(f"   {i}. Searching: {q}")
        try:
            # max_results=1 forces a short response
            search_result = tavily.search(query=q, search_depth="basic", max_results=1)

            if search_result['results']:
                content = search_result['results'][0]['content'][:600] # Limit char count
                q_and_a_results.append(f"Question: {q}\nFound Info: {content}")
            else:
                q_and_a_results.append(f"Question: {q}\nFound Info: No data.")
        except:
             q_and_a_results.append(f"Question: {q}\nFound Info: Search Failed.")

    return "\n---\n".join(q_and_a_results)

def step_3_summarize(input_text, research_data, mode):
    prompt = f"Summarize user input '{input_text[:5000]}' combined with search data '{research_data}' into a Markdown report."
    response = model.generate_content(prompt)
    return response.text

In [51]:
# --- MAIN LOOP ---

def main():
    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        if choice == '1':
            mode = "topic"
            input_data = input("\nEnter Research Topic: ")
            label = input_data

        elif choice == '2':
            mode = "pdf"
            print("\n⬆️ Upload PDF now:")
            uploaded = files.upload()
            if uploaded:
                filename = next(iter(uploaded))
                input_data = extract_pdf_text(uploaded[filename])
                label = f"PDF: {filename}"
            else:
                print("No file uploaded.")
                continue

        if input_data:
            # --- EXECUTE 3-STEP PROCESS ---

            # 1. Generate Questions
            questions = step_1_generate_questions(input_data, mode)
            print(f"   -> Plan: {questions}")

            # 2. Retrieve Responses
            raw_data = step_2_get_responses(questions)

            # 3. Summarize
            final_report = step_3_summarize(input_data, raw_data, mode)

            print("\n" + "="*20 + " 🎯 FINAL REPORT: " + "="*20 + "\n")
            display(Markdown(final_report))

            # Save to History
            memory.save_entry(label, mode, final_report)

if __name__ == "__main__":
    main()


=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 2

⬆️ Upload PDF now:


Saving 2511.17546v1.pdf to 2511.17546v1 (11).pdf

Step 1: Generating 3 sub-questions (Mode: pdf)...
   -> Plan: ['Latest evaluated nuclear quadrupole moments for transition elements with Z between 23 and 30.', 'Experimental progress in using cryogenic microcalorimeters for muonic x-ray spectroscopy.', 'Advances in ab initio calculations of electric field gradients for open d-shell atoms like copper and cobalt.']
Step 2: Retrieving short responses from the web...
   1. Searching: Latest evaluated nuclear quadrupole moments for transition elements with Z between 23 and 30.
   2. Searching: Experimental progress in using cryogenic microcalorimeters for muonic x-ray spectroscopy.
   3. Searching: Advances in ab initio calculations of electric field gradients for open d-shell atoms like copper and cobalt.

🎯 FINAL REPORT:


Of course. Here is a summary of the provided research paper, incorporating the context from your search data, presented in a Markdown report format.

---

### **Summary of Research Paper: Measuring Quadrupole Moments with Muonic Atoms**

This report summarizes the scientific paper titled "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms" by S. Rathi et al.

#### **Core Idea**

The paper proposes a novel experimental method to significantly improve the accuracy of a fundamental nuclear property—the electric quadrupole moment (Q)—for light transition elements (atomic numbers Z=23 to 30, e.g., Vanadium to Zinc). The current limitation in measuring Q is the high uncertainty in theoretical calculations of the electric field gradient (EFG) for these complex atoms. The authors' new approach bypasses this problem by using **muonic atoms** (where a heavy muon replaces an electron) and measuring the tiny energy splitting (Lamb shift) in their x-ray emissions with ultra-precise **cryogenic microcalorimeters**. This method is expected to reduce the uncertainty in the reference quadrupole moments by up to an order of magnitude.

#### **Key Points**

*   **The Problem:** The electric quadrupole moment (Q) reveals the shape of an atomic nucleus (its deviation from a perfect sphere). It is typically found by measuring a hyperfine splitting constant (`B`) and dividing by a calculated electric field gradient (`Vzz`). For transition metals with their complex open d-shell electron structures, calculating `Vzz` accurately is extremely difficult, which limits the precision of the final Q value.
*   **The Proposed Solution:** The researchers propose creating muonic atoms. A muon is 200 times heavier than an electron and orbits much closer to the nucleus. This makes its energy levels highly sensitive to the nucleus's shape but largely insensitive to the complex outer electron cloud. By measuring the energy splitting of the muon's `2s-2p` levels (the Lamb shift), they can extract the quadrupole moment directly, avoiding the difficult `Vzz` calculation.
*   **Enabling Technology:** The x-ray signals from these transitions are very weak and their energy splittings are tiny. The experiment's feasibility relies on using **cryogenic microcalorimeters**, a type of detector with both the high efficiency and the excellent energy resolution needed to precisely measure these specific x-rays.
*   **Expected Outcome:** According to the authors' simulations, this method could determine the reference quadrupole moment for an element with up to **10 times better accuracy** than current values.
*   **Scientific Impact:** A highly accurate "reference" quadrupole moment (Q_ref) for one stable isotope allows scientists to determine the Q values for an entire chain of related isotopes (including rare and unstable ones) with the same high precision. These improved values serve as critical benchmarks for testing and refining theories of nuclear structure and quantum chemistry.

#### **Context from Provided Search Data**

The supplementary search information you provided fits directly with the paper's context:

1.  **Relevance to Transition Elements (Z 23-30):** Your first search hit confirms the paper's focus. The paper explicitly presents a new method to measure the **"quadrupole moments for transition elements with Z between 23 and 30."**
2.  **Use of Cryogenic Microcalorimeters:** The second search hit shows that **"cryogenic microcalorimeters"** are a key enabling technology in this field of physics, reinforcing the authors' claim that their proposed experimental setup is feasible and at the forefront of measurement science.
3.  **The Calculation Challenge:** The third hit highlights that **"ab initio calculations of electric field gradients (EFGs)"** for open d-shell atoms are a complex and active area of research. This confirms the primary motivation of the paper: to develop an experimental method that can bypass these difficult theoretical calculations and achieve a more direct, accurate measurement.

---
**Publication Details:**
*   **Title:** Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms
*   **Authors:** S. Rathi, K. von Schoeler, P. Indelicato, and B. Ohayon
*   **Source:** arXiv:2511.17546v1 [physics.atom-ph]

✅ Research saved to History.

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-26 14:08   | pdf      | PDF: 2511.17546v1 (7).pdf
2    | 2025-11-26 14:10   | pdf      | PDF: 2511.17546v1 (8).pdf
3    | 2025-11-26 14:53   | pdf      | PDF: 2511.17546v1 (11).pdf

Enter ID to view full report (or Press Enter to go back): 3

🔁 RECALLING REPORT FOR: PDF: 2511.17546v1 (11).pdf
--------------------------------------------------


Of course. Here is a summary of the provided research paper, incorporating the context from your search data, presented in a Markdown report format.

---

### **Summary of Research Paper: Measuring Quadrupole Moments with Muonic Atoms**

This report summarizes the scientific paper titled "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms" by S. Rathi et al.

#### **Core Idea**

The paper proposes a novel experimental method to significantly improve the accuracy of a fundamental nuclear property—the electric quadrupole moment (Q)—for light transition elements (atomic numbers Z=23 to 30, e.g., Vanadium to Zinc). The current limitation in measuring Q is the high uncertainty in theoretical calculations of the electric field gradient (EFG) for these complex atoms. The authors' new approach bypasses this problem by using **muonic atoms** (where a heavy muon replaces an electron) and measuring the tiny energy splitting (Lamb shift) in their x-ray emissions with ultra-precise **cryogenic microcalorimeters**. This method is expected to reduce the uncertainty in the reference quadrupole moments by up to an order of magnitude.

#### **Key Points**

*   **The Problem:** The electric quadrupole moment (Q) reveals the shape of an atomic nucleus (its deviation from a perfect sphere). It is typically found by measuring a hyperfine splitting constant (`B`) and dividing by a calculated electric field gradient (`Vzz`). For transition metals with their complex open d-shell electron structures, calculating `Vzz` accurately is extremely difficult, which limits the precision of the final Q value.
*   **The Proposed Solution:** The researchers propose creating muonic atoms. A muon is 200 times heavier than an electron and orbits much closer to the nucleus. This makes its energy levels highly sensitive to the nucleus's shape but largely insensitive to the complex outer electron cloud. By measuring the energy splitting of the muon's `2s-2p` levels (the Lamb shift), they can extract the quadrupole moment directly, avoiding the difficult `Vzz` calculation.
*   **Enabling Technology:** The x-ray signals from these transitions are very weak and their energy splittings are tiny. The experiment's feasibility relies on using **cryogenic microcalorimeters**, a type of detector with both the high efficiency and the excellent energy resolution needed to precisely measure these specific x-rays.
*   **Expected Outcome:** According to the authors' simulations, this method could determine the reference quadrupole moment for an element with up to **10 times better accuracy** than current values.
*   **Scientific Impact:** A highly accurate "reference" quadrupole moment (Q_ref) for one stable isotope allows scientists to determine the Q values for an entire chain of related isotopes (including rare and unstable ones) with the same high precision. These improved values serve as critical benchmarks for testing and refining theories of nuclear structure and quantum chemistry.

#### **Context from Provided Search Data**

The supplementary search information you provided fits directly with the paper's context:

1.  **Relevance to Transition Elements (Z 23-30):** Your first search hit confirms the paper's focus. The paper explicitly presents a new method to measure the **"quadrupole moments for transition elements with Z between 23 and 30."**
2.  **Use of Cryogenic Microcalorimeters:** The second search hit shows that **"cryogenic microcalorimeters"** are a key enabling technology in this field of physics, reinforcing the authors' claim that their proposed experimental setup is feasible and at the forefront of measurement science.
3.  **The Calculation Challenge:** The third hit highlights that **"ab initio calculations of electric field gradients (EFGs)"** for open d-shell atoms are a complex and active area of research. This confirms the primary motivation of the paper: to develop an experimental method that can bypass these difficult theoretical calculations and achieve a more direct, accurate measurement.

---
**Publication Details:**
*   **Title:** Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms
*   **Authors:** S. Rathi, K. von Schoeler, P. Indelicato, and B. Ohayon
*   **Source:** arXiv:2511.17546v1 [physics.atom-ph]


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋
